## Baseline Scenario

Keep a running average for throughput (sensors and users) and a running average for age-of-info. 
- If the age-of-info is larger than threshold double the sensor BW and CPU. 
- If the throughput is less than threshold double BW/CPU for users.


In [ ]:
import gymnasium as gym
import matplotlib.pyplot as plt

# predefined smart city scenario
from mobile_env.scenarios.smart_city import MComSmartCity

In [ ]:
env = gym.make("mobile-smart_city-smart_city_handler-v0", render_mode="rgb_array")

print(f"\nSmart city environment with {env.NUM_USERS} users, {env.NUM_SENSORS} sensors and {env.NUM_STATIONS} cells.")

In [ ]:
from IPython import display
import matplotlib.pyplot as plt

# Parameters
aoi_threshold = 10.0  # Replace with your desired threshold
throughput_threshold = 50.0  # Replace with your desired throughput threshold
running_avg_aoi = 0.0
running_avg_throughput_ue = 0.0
running_avg_throughput_sensor = 0.0
aoi_alpha = 0.1  # Weight for exponential moving average
throughput_alpha = 0.1  # Weight for exponential moving average

# Environment reset
done = False
obs, info = env.reset()

# Run simulation for 100 steps
for t in range(100):
    # Perform a dummy action (you can replace it with learned actions or dynamic policies)
    dummy_action = (0.50, 0.50)
    obs, reward, terminated, truncated, info = env.step(dummy_action)

    # Get metrics from the environment
    scalar_results, _, _, _, _ = env.monitor.load_results()
    current_aoi = scalar_results['age-of-info'].iloc[-1]  # Replace with correct metric name
    current_throughput_ue = scalar_results['total throughput ue'].iloc[-1]  # Replace with correct metric name
    current_throughput_sensor = scalar_results['total throughput sensor'].iloc[-1]  # Replace with correct metric name

    # Update running averages using exponential moving average
    running_avg_aoi = (1 - aoi_alpha) * running_avg_aoi + aoi_alpha * current_aoi
    running_avg_throughput_ue = (1 - throughput_alpha) * running_avg_throughput_ue + throughput_alpha * current_throughput_ue
    running_avg_throughput_sensor = (1 - throughput_alpha) * running_avg_throughput_sensor + throughput_alpha * current_throughput_sensor

    # Adjust resources based on thresholds
    if running_avg_aoi > aoi_threshold:
        print(f"Time {t}: Doubling Sensor BW and CPU due to high AoI ({running_avg_aoi:.2f})")
        for sensor in env.sensors.values():
            sensor.bw *= 2
            sensor.cpu *= 2

    if running_avg_throughput_ue < throughput_threshold:
        print(f"Time {t}: Doubling User BW and CPU due to low throughput ({running_avg_throughput_ue:.2f})")
        for ue in env.users.values():
            ue.bw *= 2
            ue.cpu *= 2

    # Check termination condition
    if terminated or truncated:
        break

    # Render the environment
    plt.imshow(env.render())
    display.display(plt.gcf())
    display.clear_output(wait=True)

# Close the plot after simulation
plt.close()


In [ ]:
scalar_results, kpi_results, ue_results, bs_results, ss_results = env.monitor.load_results()

scalar_results

In [ ]:
kpi_results

In [ ]:
ue_results

In [ ]:
ss_results

In [ ]:
bs_results

In [ ]:
data = kpi_results.agg(['min', 'max', 'mean'])
data

In [ ]:
data = scalar_results.agg(['min', 'max', 'mean'])
data

In [ ]:
data = ue_results.reset_index()
data = data.groupby('UE ID').agg(['mean', 'min', 'max'])
data = data.drop('Time Step', axis='columns')
data

In [ ]:
data = ss_results.reset_index()
data = data.groupby('Sensor ID').agg(['mean', 'min', 'max'])
data = data.drop('Time Step', axis='columns')
data

In [ ]:
data = bs_results.reset_index()
data = data.groupby('BS ID').agg(['mean', 'min', 'max'])
data = data.drop('Time Step', axis='columns')
data

In [ ]:
import matplotlib.pyplot as plt

def plot_kpi_metrics():
    # Plot kpi metrics
    for metric in kpi_results.columns:
        plt.figure(figsize=(12, 8))
        kpi_results[metric].plot()
        plt.title(f"{metric}")
        plt.xlabel("Time Step")
        plt.ylabel("Value")
        plt.grid(True)
        plt.show()

plot_kpi_metrics()

In [ ]:
import pandas as pd
import time

# Reset environment and initialize variables
done = False
obs, info = env.reset()
all_kpi_means = []  # List to store mean metrics for each episode
episode_infos = []

# Run for 100 time steps for 100 episodes
for episode in range(100):
    obs, info = env.reset()
    print(f"Starting Episode {episode + 1}")
    episode_start_time = time.time()  # Start timing the episode

    for t in range(100):
        # Perform dummy action
        dummy_action = (1.0, 1.0)
        obs, reward, terminated, truncated, info = env.step(dummy_action)

        # Check termination condition
        if terminated or truncated:
            break
    
    # Calculate and store the episode duration
    episode_duration = time.time() - episode_start_time
    print(f"Completed Episode {episode + 1}: {episode_duration}")

    # Load results after the episode
    _, kpi_results, _, _, _ = env.monitor.load_results()

    # Compute mean of metrics over the episode
    kpi_mean = kpi_results.mean()  # Calculate mean for each metric
    all_kpi_means.append(kpi_mean)

    # Add episode-level information to the list
    episode_info = {
        "Episode": episode + 1,
        "Total Reward": info.get("episode reward"),
        "Queue Size Transferred Jobs UE": info.get("bs trans. ue"),
        "Queue Size Transferred Jobs Sensor": info.get("bs trans. ss"),
        "Queue Size Accomplished Jobs UE": info.get("bs accomp. us"),
        "Queue Size Accomplished Jobs Sensor": info.get("bs accomp. ss"),
    }
    episode_infos.append(episode_info)

    print(f"Mean Reward for Episode: {episode_info['Total Reward']/100:.2f}")


# Convert all_kpi_means and episode_infos to DataFrames
episode_infos_df = pd.DataFrame(episode_infos)
all_kpi_means_df = pd.DataFrame(all_kpi_means)

# Display mean KPI metrics for each episode
print(episode_infos_df)
print(all_kpi_means_df)

# Save results to CSV files
episode_infos_df.to_csv("episode_infos.csv", index=False)
all_kpi_means_df.to_csv("kpi_means_per_episode.csv", index=False)

In [ ]:
# Plot each metric in a separate plot
required_metrics = ["reward", "total throughput ue", "total throughput sensor", "total delayed packets", "total aori", "total aosi"]

for metric in required_metrics:
    plt.figure(figsize=(10, 6))
    plt.plot(all_kpi_means_df.index, all_kpi_means_df[metric], marker='o', label=metric)
    plt.title(f"Mean {metric} Over Episodes", fontsize=16)
    plt.xlabel("Episode", fontsize=14)
    plt.ylabel("Value", fontsize=14)
    plt.grid(True)
    plt.legend()
    plt.show()

In [ ]:
import numpy as np

# Function to calculate and plot the CDF for each metric
def plot_cdf_metrics(metrics_df, metrics_list):
    for metric in metrics_list:
        plt.figure(figsize=(10, 6))
        
        # Get the values for the metric
        metric_values = metrics_df[metric].dropna().values  # Drop NaN values
        
        # Sort the values and compute the CDF
        sorted_values = np.sort(metric_values)
        cdf = np.arange(1, len(sorted_values) + 1) / len(sorted_values)
        
        # Plot the CDF
        plt.plot(sorted_values, cdf, marker='o', label=f"CDF of {metric}")
        plt.title(f"CDF of {metric}", fontsize=16)
        plt.xlabel(f"{metric} Value", fontsize=14)
        plt.ylabel("Cumulative Probability", fontsize=14)
        plt.grid(True)
        plt.legend()
        plt.show()

# Metrics to plot
required_metrics = ["reward", "total throughput ue", "total throughput sensor", "total delayed packets", "total aori", "total aosi"]

# Call the CDF plot function
plot_cdf_metrics(all_kpi_means_df, required_metrics)
